# odc-stac tutorial

In this tutorial we will use Python libraries to find and load land cover data from the freely available [Impact Observatory Annual Land Use Land Cover](https://planetarycomputer.microsoft.com/dataset/io-lulc-annual-v02) product.
After loading the data, we will export it as a cloud-optimised GeoTiff.
This will allow you to further view or work with the data in GIS software and other tools.

During the tutorial, we will:

* Specify our search in terms of:

  * what (data provider and satellite)
  * where (area of interest)
  * when (date range)
* Use `pystac-client` to connect to a Spatio-Temporal Asset Catalog (STAC) 
  endpoint and search for data matching our what, where, and when
* Use `odc-stac` to load the matching data into memory
* Visualise and export the data

## Requirements
Please keep a copy of the [tutorial instructions](https://opendatacube.readthedocs.io/en/latest/tutorials/odc-stac.html) open.

Follow the instructions for each labelled section. 
When you have finished entering code into a cell, press `Shift+Enter` on your keyboard to run the cell.

## Python imports

In this section, you will import the three packages that the notebook needs to run:

* `geopandas` for loading an area of interest from a file
* `odc.stac` for loading satellite data
* `pystac-client` for querying catalogs of satellite data

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.


## Set up query parameters

In this section of the tutorial, you will specify:

* The area you want to load data for
* The date range you want to load data for
* The catalog you want to load data from

Follow the instructions for each section.

### Area of interest

In this section, you will:

* Load the area of interest from a supplied `geojson` file called `aoi.geojson`
* Extract the geometry of the area of interest for later use

The area of interest covers part of Wilkinkarra (Lake Mackay) in Australia.
It is one of Australia's large salt lakes. 

![A satellite image of Wilkinkarra, with the area of interest shown as a yellow bounding box.](images/wilkinkarra_aoi.png "Wilkinkarra area of interest")

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.

### Date range

In this section, you will:
* Specify the start date for the search as `"2021-12-24"`
* Specify the end date for the search as `"2021-12-26"`
* Store these as a pair of dates for later use

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.

### Catalog and measurements

In this section, you will:
* Specify the STAC catalog to search as `"https://earth-search.aws.element84.com/v1/"`
* Specify the collection to search as `["sentinel-2-l2a"]`
* Specify the bands to load as `["red", "green", "blue"]`


Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.


## Connect to catalog and find items

In this section, you will:
* Create a client

This step returns STAC items that match the query, each of which points to a Cloud Optimised GeoTIFF (COG) that contains the relevant data

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.

### Search for items

`pystac-client` is used to search for items that match the query. The search function has a few arguments:

- `collections`: a list of collections to search
- `intersects`: an area of interest to search. Any item intersecting the geometry will be returned.
- `datetime`: a date range to search

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.

In [ ]:
# Search for items matching the query
items = stac_client.search(
    collections=collections_query,
    intersects=aoi_geometry,
    datetime=date_query,
).item_collection()

print(f"Found {len(items)} items")

Once returned, it is possible to view relevant metadata for all discovered items.

In [ ]:
items

## Load items with odc-stac

Once the location of the items is known, `odc-stac` can load these items. The load function has a few arguments:

- `items`: a list of STAC items to load
- `bands`: a list of assets to load for each item
- `geopolygon`: the area of interest to load the data for
- `crs`: the coordinate reference system to use. `utm` will provide the most appropriate UTM projection
- `resolution`: the resolution to load the items at, in the same units as the `crs`. If using `utm` the unit is metres
- `groupby`: using `solar_day` here will merge all items captured on the same solar day into one image


```
ds_filtered = load(
    items,
    bands=bands_query,
    geopolygon=aoi_geometry,
    crs="utm",
    resolution=30,
    groupby="solar_day",
)

ds_filtered
```

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.


## Visualise loaded data

If an `xarray` has been loaded using the Open Data Cube ecosystem, it comes with an `xarray` extension `xarray.odc` that allows you to access additional geospatial metadata and functions.

Learn more at [odc-geo](https://odc-geo.readthedocs.io/en/latest/intro-xr.html).

```
# Get a single image at the first time step, and remove the time dimension
image = ds_filtered.isel(time=0).squeeze()

image
```

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.

```
# Create an RGB image using the .odc. extension
visualisation = image.odc.to_rgba()

# Display the RGB image on a map
visualisation.odc.explore()
```

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.

## Export loaded data

The `.odc` extension is also useful for saving out data as Cloud Optimised GeoTIFFs (COGs).

```
visualisation.odc.write_cog(
    "sentinel2_example.tif",
    overwrite=True
)
```

Type the code as shown in the instructions.
When you have finished, run the cell by pressing `Shift+Enter` on your keyboard.


## Tutorial Complete!

Try loading your saved COG in QGIS or another GIS program.